# Examine enrichment in psychiatric disorder TWAS and DEGs

In [1]:
import functools
import numpy as np
import pandas as pd
from os import environ
from pybiomart import Dataset
from scipy.stats import fisher_exact
from statsmodels.stats.multitest import multipletests

## Configuration

In [2]:
environ['NUMEXPR_MAX_THREADS'] = '32'

config = {
    "dlpfc_file": "/ceph/users/jbenja13/phase3_paper/phase2/extract_de/_m/dlpfc_diffExpr_szVctl_full.txt",
    "caud8_file": "/ceph/projects/v4_phase3_paper/analysis/differential_expression/_m/genes/diffExpr_szVctl_full.txt",
    "hippo_file": "/ceph/users/jbenja13/phase3_paper/phase2/extract_de/_m/hippo_diffExpr_szVctl_full.txt",
    'cmc_file': '/ceph/projects/v3_phase3_paper/inputs/cmc/_m/CMC_MSSM-Penn-Pitt_DLPFC_mRNA_IlluminaHiSeq2500'+\
    '_gene-adjustedSVA-differentialExpression-includeAncestry-DxSCZ-DE.tsv',
    'gandal_de_file': "/ceph/users/jbenja13/psychENCODE/expression_results/_m/gandal2018_psychENCODE_DE_results.xlsx",
    'twas_asd_file': '/ceph/users/jbenja13/psychENCODE/_m/psychENCODE_twas_asd.csv',
    'twas_sz_file': '/ceph/users/jbenja13/psychENCODE/_m/psychENCODE_twas_sz.csv',
    'twas_bd_file': '/ceph/users/jbenja13/psychENCODE/_m/psychENCODE_twas_bd.csv',
    'twas_bs_file': '/ceph/projects/v4_phase3_paper/analysis/twas/public_twas_comp/_m/TWAS_gene_tissue_summary.csv'
}

## Functions

### Cached

In [3]:
@functools.lru_cache()
def get_database():
    dataset = Dataset(name="hsapiens_gene_ensembl", 
                      host="http://www.ensembl.org",
                      use_cache=True)
    db = dataset.query(attributes=["ensembl_gene_id", 
                                   "external_gene_name", 
                                   "entrezgene_id"], 
                       use_attr_names=True).dropna(subset=['entrezgene_id'])
    return db


@functools.lru_cache()
def get_deg(fn):
    return pd.read_csv(fn, sep='\t')


@functools.lru_cache()
def get_sig_deg(fn):
    return get_deg(fn)[(get_deg(fn)["adj.P.Val"] < 0.05)]


@functools.lru_cache()
def get_gandal_deg():
    return pd.read_excel(config["gandal_de_file"], sheet_name="DGE")


@functools.lru_cache()
def get_eGenes(tissue):
    df = pd.read_csv("../../summary_table/_m/BrainSeq_sexGenotypes_4features_3regions.txt.gz", sep='\t')
    df = df[(df["Type"] == "Gene") & (df["Tissue"] == tissue)]\
        .drop(["variant_id", "lfsr"], axis=1).drop_duplicates()
    df["ensemblID"] = df.gencodeID.str.replace("\\..*", "", regex=True)
    return df


@functools.lru_cache()
def get_bs_twas():
    return pd.read_csv(config["twas_bs_file"])


@functools.lru_cache()
def get_gandal_twas(fn):
    df = pd.read_csv(fn)
    return df[(df["TWAS.Bonferroni"] < 0.05)]

### Simple

In [4]:
def fet(a, b):
    """
    Calculates Fisher's Exact test (fet) with sets a and b in universe u.
    Inputs are sets.
    """
    u = set(get_database().ensembl_gene_id)
    a = set(a); b = set(b)
    yes_a = u.intersection(a)
    yes_b = u.intersection(b)
    no_a = u - a
    no_b = u - b
    m = [[len(yes_a.intersection(yes_b)), len(no_a.intersection(yes_b))], 
         [len(yes_a.intersection(no_b)), len(no_a.intersection(no_b))]]
    return fisher_exact(m)

## Load eGenes for sex-interacting analysis

In [5]:
caudate = set(get_eGenes("Caudate").ensemblID)
dlpfc = set(get_eGenes("DLPFC").ensemblID)
hippocampus = set(get_eGenes("Hippocampus").ensemblID)

## Differential expression

### BrainSeq SZ case vs control

In [6]:
bs_caudate_degs = set(get_sig_deg(config["caud8_file"]).ensemblID)
bs_dlpfc_degs = set(get_sig_deg(config["dlpfc_file"]).ensemblID)
bs_hippo_degs = set(get_sig_deg(config["hippo_file"]).ensemblID)

### CommonMind SZ, DLPFC

In [7]:
cmc_dlpfc_degs = set(get_sig_deg(config["cmc_file"]).genes)

### PsychENCODE (Gandal)

In [8]:
psy_sz = set(get_gandal_deg()[(get_gandal_deg()["SCZ.fdr"] < 0.05)].ensembl_gene_id)
psy_asd = set(get_gandal_deg()[(get_gandal_deg()["ASD.fdr"] < 0.05)].ensembl_gene_id)
psy_bd = set(get_gandal_deg()[(get_gandal_deg()["BD.fdr"] < 0.05)].ensembl_gene_id)

## TWAS

### BrainSeq

In [9]:
bs_caudate_twas = set(get_bs_twas()[(get_bs_twas()["Caudate_FDR"] < 0.05)].Geneid)
bs_dlpfc_twas = set(get_bs_twas()[(get_bs_twas()["DLPFC_FDR"] < 0.05)].Geneid)
bs_hippo_twas = set(get_bs_twas()[(get_bs_twas()["HIPPO_FDR"] < 0.05)].Geneid)

### PsychENCODE

In [10]:
psy_asd_twas = set(get_gandal_twas(config["twas_asd_file"]).GeneID)
psy_sz_twas = set(get_gandal_twas(config["twas_sz_file"]).GeneID)
psy_bd_twas = set(get_gandal_twas(config["twas_bd_file"]).ID)

## Enrichment analysis

In [11]:
egenes_dict = {"Caudate": caudate, "DLPFC": dlpfc, "Hippocampus": hippocampus}
comp_dict = {"BS_Caudate_DEG": bs_caudate_degs, "BS_DLPFC_DEG": bs_dlpfc_degs, 
             "BS_Hippocampus_DEG": bs_hippo_degs, "CMC_DLPFC_DEG": cmc_dlpfc_degs, 
             "PSY_SZ_DEG": psy_sz, "PSY_ASD_DEG": psy_asd, "PSY_BD_DEG": psy_bd,
             "BS_Caudate_TWAS": bs_caudate_twas, "BS_DLPFC_TWAS": bs_dlpfc_twas, 
             "BS_Hippocampus_TWAS": bs_hippo_twas, "PSY_SZ_TWAS": psy_sz_twas, 
             "PSY_ASD_TWAS": psy_asd_twas, "PSY_BD_TWAS": psy_bd_twas}
comp_list = ["BS_Caudate_DEG", "BS_DLPFC_DEG", "BS_Hippocampus_DEG", 
             "CMC_DLPFC_DEG", "PSY_SZ_DEG", "PSY_ASD_DEG", "PSY_BD_DEG", 
             "BS_Caudate_TWAS", "BS_DLPFC_TWAS", "BS_Hippocampus_TWAS", 
             "PSY_SZ_TWAS", "PSY_ASD_TWAS", "PSY_BD_TWAS"]

or_lt = []; pval_lt = []; tissue_lt = []; comparison_lt = [];
for tissue in ["Caudate", "DLPFC", "Hippocampus"]:
    for comp in comp_list:
        oddratio, pvals = fet(egenes_dict[tissue], comp_dict[comp])
        or_lt.append(oddratio); pval_lt.append(pvals);
        tissue_lt.append(tissue); comparison_lt.append(comp)
fdr = multipletests(pval_lt, method='fdr_bh')[1]
dt = pd.DataFrame({"Tissue": tissue_lt, "Comparison": comparison_lt, "OR": or_lt, 
                   "P-value": pval_lt, "FDR": fdr})

In [12]:
dt[(dt["FDR"] < 0.05)]

,Tissue,Comparison,OR,P-value,FDR
0,Caudate,BS_Caudate_DEG,2.756170,1.392999e-25,5.432696e-24
1,Caudate,BS_DLPFC_DEG,2.349303,4.310847e-03,8.848580e-03
3,Caudate,CMC_DLPFC_DEG,1.812393,1.662972e-02,2.687730e-02
4,Caudate,PSY_SZ_DEG,2.063669,7.475636e-18,5.830996e-17
5,Caudate,PSY_ASD_DEG,2.491460,1.093601e-13,6.092922e-13
6,Caudate,PSY_BD_DEG,1.532679,1.004704e-02,1.781066e-02
7,Caudate,BS_Caudate_TWAS,2.500256,1.529816e-06,5.966284e-06
8,Caudate,BS_DLPFC_TWAS,2.251763,3.043822e-03,6.982886e-03
9,Caudate,BS_Hippocampus_TWAS,2.775863,2.302466e-03,5.986412e-03
11,Caudate,PSY_ASD_TWAS,10.722030,2.261958e-02,3.392937e-02


In [13]:
dt.to_csv("clincial_phenotypes_enrichment_analysis_3brainRegions.tsv", sep='\t', index=False)